### file path:

C:\Users\Computer\Desktop\nnc_updated_ah\gc\books\book-academic-jana-siksha-2058

### how to preprocess this type of data to train postagging model using LSTM algorithms. explain step by step procedure with code

Preprocessing textual data for training a part-of-speech tagging model using LSTM algorithms involves several steps. I'll provide you with a step-by-step procedure along with Python code snippets for each step. For this example, I'll assume you're using the NLTK library for natural language processing tasks.

# 1. Data Loading and Parsing:

Load the XML data and extract the text content for preprocessing.

In [ ]:
import xml.etree.ElementTree as ET

# Load the XML data
tree = ET.parse('book-academic-jana-siksha-2058.xml')
root = tree.getroot()

# Extract text content from <w> tags
text_data = ' '.join([w.text for w in root.iter('w')])
# text_data


# 2. Text Tokenization:
Tokenize the text data into words.

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Tokenize the text data
tokens = word_tokenize(text_data)
# tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 3. Remove Special Tags:
Remove any special tags like `< cesDoc >`, `< cesHeader >`, etc., and keep only the actual word tokens.

In [ ]:
tokens = [token for token in tokens if not token.startswith('<')]
# tokens

# 4. Text Normalization:
Normalize the text by converting it to lowercase.

In [ ]:
normalized_tokens = [token.lower() for token in tokens]


# 5. Create Vocabulary and Index Mapping:
Create a vocabulary and map words to unique indices.

In [ ]:
vocab = set(normalized_tokens)
word_to_index = {word: index + 1 for index, word in enumerate(vocab)}  # Start index from 1
index_to_word = {index: word for word, index in word_to_index.items()}


# 6. Convert Words to Indices:
Convert tokens to corresponding indices.

In [ ]:
indexed_data = [word_to_index[word] for word in normalized_tokens]


# 7. Generate Training Examples:
Generate input-output pairs for training the LSTM model. The input is a sequence of words, and the output is the corresponding part-of-speech tags.

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk import pos_tag

# Perform part-of-speech tagging on the original text
pos_tags = pos_tag(tokens)

# Create a mapping from part-of-speech tags to unique indices
tag_set = set(tag for word, tag in pos_tags)
tag_to_index = {tag: index for index, tag in enumerate(tag_set)}

# Generate training examples
sequence_length = 10  # You can adjust this value as needed
X = [indexed_data[i:i+sequence_length] for i in range(len(indexed_data) - sequence_length)]
y = [tag_to_index[pos_tags[i + sequence_length][1]] for i in range(len(indexed_data) - sequence_length)]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# 8. Padding Sequences:
Pad sequences to a fixed length for training.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_padded = pad_sequences(X, maxlen=sequence_length, padding='post')
X_padded

array([[4491, 4548, 5103, ..., 4785, 5698, 3572],
       [4548, 5103, 2933, ..., 5698, 3572, 3678],
       [5103, 2933, 6642, ..., 3572, 3678, 1165],
       ...,
       [1454, 2304, 4482, ..., 2180, 4183, 5029],
       [2304, 4482, 5426, ..., 4183, 5029, 5016],
       [4482, 5426, 5029, ..., 5029, 5016, 2259]], dtype=int32)

# 9. One-Hot Encoding:
Convert part-of-speech tags to one-hot encoded vectors.

 `np.eye` function is used to create a 2D identity matrix with ones on the diagonal and zeros elsewhere.

`len(tag_set)` : Number of rows in the output matrix

In [ ]:
import numpy as np

y_encoded = np.eye(len(tag_set))[y]


In [ ]:
y_encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 2nd part


# 10. Train Word2Vec Model:
Train a Word2Vec model on your tokenized text data. You can use the `Word2Vec` class from the `gensim` library.

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
w2v_model = Word2Vec(sentences=[normalized_tokens], vector_size=100, window=5, min_count=1, sg=0)


# 11. Get Word Vectors:
Once you have the trained Word2Vec model, you can obtain the dense vectors for words.

In [ ]:

# Get vector for a specific word
word_vector = w2v_model.wv['म']

# Get vectors for a list of words
word_vectors = [w2v_model.wv[word] for word in normalized_tokens]


# 12. Padding Word Vectors:
If you want to maintain a fixed sequence length, you'll need to pad the word vectors similar to what you did with the integer indices in the previous example.

In [ ]:
X_word2vec = [word_vectors[i:i+sequence_length] for i in range(len(word_vectors) - sequence_length)]
X_word2vec_padded = pad_sequences(X_word2vec, maxlen=sequence_length, padding='post')


Now you have the word vectors from the Word2Vec model represented as dense vectors ready for training.

# 13. Data Splitting:

divide dataset into a training set and a test set using the common 80-20 split.

training the model on the training set, it evaluates the model's performance on the test set using the evaluate method.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_word2vec_padded, y_encoded, test_size=0.2, random_state=42)

# 13. Build and Train LSTM Model:
Next, you can build an LSTM model to predict part-of-speech tags using the word vectors as input.

In [ ]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense

# # Build LSTM model
# model = Sequential()
# model.add(LSTM(units=128, input_shape=(sequence_length, 100)))  # Assuming vector_size=100 in Word2Vec
# model.add(Dense(len(tag_set), activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(X_word2vec_padded, y_encoded, epochs=10, batch_size=32, validation_split=0.2)


This code builds and trains an LSTM model using the Word2Vec word vectors as input. You might need to adjust hyperparameters, such as the number of LSTM units, the vector size in the Word2Vec model, and the training parameters, according to your specific problem and dataset.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Build LSTM model
model = Sequential()
model.add(LSTM(units=128, input_shape=(sequence_length, 100)))  # Assuming vector_size=100 in Word2Vec
model.add(Dense(len(tag_set), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training set
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

Epoch 1/10
908/908 [==============================] - 15s 15ms/step - loss: 0.7239 - accuracy: 0.8738 - val_loss: 0.6688 - val_accuracy: 0.8768
Epoch 2/10
908/908 [==============================] - 13s 14ms/step - loss: 0.6718 - accuracy: 0.8748 - val_loss: 0.6649 - val_accuracy: 0.8768
Epoch 3/10
908/908 [==============================] - 13s 14ms/step - loss: 0.6710 - accuracy: 0.8748 - val_loss: 0.6662 - val_accuracy: 0.8768
Epoch 4/10
908/908 [==============================] - 14s 15ms/step - loss: 0.6703 - accuracy: 0.8748 - val_loss: 0.6611 - val_accuracy: 0.8768
Epoch 5/10
908/908 [==============================] - 14s 16ms/step - loss: 0.6698 - accuracy: 0.8748 - val_loss: 0.6594 - val_accuracy: 0.8768
Epoch 6/10
908/908 [==============================] - 13s 14ms/step - loss: 0.6683 - accuracy: 0.8748 - val_loss: 0.6592 - val_accuracy: 0.8768
Epoch 7/10
908/908 [==============================] - 13s 14ms/step - loss: 0.6656 - accuracy: 0.8748 - val_loss: 0.6626 - val_accuracy:

# All the above step are together


In [3]:
!pip install tensorflow

In [4]:
!pip install nltk gensim scikit-learn


In [5]:
!pip install --upgrade tensorflow nltk gensim scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: scikit-learn
    

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

import xml.etree.ElementTree as ET

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag




# Step 1: Data Preprocessing
# Load and preprocess your XML data
### Load the XML data
tree = ET.parse('book-academic-jana-siksha-2058.xml')
root = tree.getroot()

### Extract text content from <w> tags
text_data = ' '.join([w.text for w in root.iter('w')])



# Tokenize and normalize the text data
### Tokenize the text data
tokens = word_tokenize(text_data)

### remove special tags
tokens = [token for token in tokens if not token.startswith('<')]

### Text normalization
normalized_tokens = [token.lower() for token in tokens]

# Create vocabulary and index mapping
vocab = set(normalized_tokens)
word_to_index = {word: index + 1 for index, word in enumerate(vocab)}  # Start index from 1
index_to_word = {index: word for word, index in word_to_index.items()}

# Convert words to indices
indexed_data = [word_to_index[word] for word in normalized_tokens]

# Define the sequence length for LSTM input
sequence_length = 10

# Step 2: Word Embedding using Word2Vec
w2v_model = Word2Vec(sentences=[normalized_tokens], vector_size=100, window=5, min_count=1, sg=0)

# Get word vectors
### Train Word2Vec model
w2v_model = Word2Vec(sentences=[normalized_tokens], vector_size=100, window=5, min_count=1, sg=0)

### Get vector for a specific word
word_vector = w2v_model.wv['म']

### Get vectors for a list of words
word_vectors = [w2v_model.wv[word] for word in normalized_tokens]

# Pad word vectors
X_word2vec = [word_vectors[i:i+sequence_length] for i in range(len(word_vectors) - sequence_length)]
X_word2vec_padded = pad_sequences(X_word2vec, maxlen=sequence_length, padding='post')
#-------------------------------------------------------------------------------------------------------------------


# Encoding Labels (e.g., one-hot encoding or label encoding):
### Perform part-of-speech tagging on the original text
pos_tags = pos_tag(tokens)

### Create a mapping from part-of-speech tags to unique indices
tag_set = set(tag for word, tag in pos_tags)
tag_to_index = {tag: index for index, tag in enumerate(tag_set)}

### generate training example
y = [tag_to_index[pos_tags[i + sequence_length][1]] for i in range(len(indexed_data) - sequence_length)]

# Encoding Labels (e.g., one-hot encoding or label encoding):
y_encoded = np.eye(len(tag_set))[y]
# -------------------------------------------------------------------------------------------------------------


# Step 3: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X_word2vec_padded, y_encoded, test_size=0.2, random_state=42)

# Step 4: Create LSTM Model
model = Sequential()
model.add(LSTM(units=128, input_shape=(sequence_length, 100)))
model.add(Dense(len(tag_set), activation='softmax'))

# Step 5: Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Step 7: Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Epoch 1/10
908/908 [==============================] - 28s 28ms/step - loss: 0.7219 - accuracy: 0.8735 - val_loss: 0.6634 - val_accuracy: 0.8768
Epoch 2/10
908/908 [==============================] - 22s 25ms/step - loss: 0.6723 - accuracy: 0.8748 - val_loss: 0.6657 - val_accuracy: 0.8768
Epoch 3/10
908/908 [==============================] - 22s 24ms/step - loss: 0.6715 - accuracy: 0.8748 - val_loss: 0.6649 - val_accuracy: 0.8768
Epoch 4/10
908/908 [==============================] - 30s 33ms/step - loss: 0.6707 - accuracy: 0.8748 - val_loss: 0.6671 - val_accuracy: 0.8768
Epoch 5/10
908/908 [==============================] - 22s 24ms/step - loss: 0.6700 - accuracy: 0.8748 - val_loss: 0.6637 - val_accuracy: 0.8768
Epoch 6/10
908/908 [==============================] - 29s 32ms/step - loss: 0.6689 - accuracy: 0.8748 - val_loss: 0.6585 - val_accuracy: 0.8768
Epoch 7/10
908/908 [==============================] - 22s 24ms/step - loss: 0.6672 - accuracy: 0.8748 - val_loss: 0.6585 - val_accuracy: